In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from datetime import datetime

In [2]:
class ReversedAnalysis:
    def __init__(self, base_format=None, df=None):
        self.base_format = deepcopy(base_format)
        self.df = deepcopy(df)
        self.base_format.drop(["Cost", "Raanana", "Target", "Avg(FQ>0)", "BB", "Sales"], axis=1, inplace=True)
        self._skip = False
        self._name = None
        
    def process_helium(self):
        '''
        Here we process file from Helium, delete extra rows, etc.
        '''
        self.df = self.df.drop(["URL", "Image URL", "Revenue", "Seller Country/Region", "Fees $", 
                                "Active Sellers #", "Ratings", "Review Count", "Images", 
                                "Review velocity", "Buy Box", "Size Tier", "Fulfillment", 
                                "Dimensions", "Weight", "Creation Date", "Category"], axis=1)
        self.df = self.df.rename({"Price $": "BB", "Product Details": "Description"}, axis=1)
        self.df = self.df.dropna(subset=["BSR", "Sales"])
        self.df["BSR"] = self.df["BSR"].apply(lambda x: x.replace(",", ""))
        self.df["BSR"] = self.df['BSR'].astype("Int64")
        self.df["Sales"] = self.df["Sales"].apply(lambda x: x.replace(",", ""))
        self.df["Sales"] = self.df['Sales'].astype("Int64")
        self.df = self.df[self.df["BSR"] < 250_000]
        self.df = self.df[self.df["Sales"] >= 20]
        self.df["Description"] = self.df["Description"].apply(lambda x: x.replace("($)", ""))
        self.df = self.df.drop(["BSR"], axis=1)
        
    def delete_extra_brands(self):
        '''
        Here we ask a user the name of the brand that we need. 
        The function will delete extra rows, promotions, mistakes from helium
        '''
        print("Please, type the name of the brand that we are analzying")
        names = input("If there are more than one brand, type it seperated with comma ',':  ")
        names = names.split(",")
        names = list(map(lambda x: x.strip(), names))
        self.df = self.df[self.df["Brand"].isin(names)]
        self.df = self.df.reset_index(drop=True)
        
    def merge_baseformat_and_df(self):
        '''
        Here we merge baseformatfile with df. This will help us to see barcodes without asins
        '''
        self.base_format = self.base_format.merge(self.df, on="ASIN", how="left").dropna(subset="Brand")
        self.base_format = self.base_format[~self.base_format["prohibited"].isin([1])] #delete restricted items
        self.base_format.reset_index(drop=True, inplace=True) 
        
    def get_missing_asins(self):
        """
        Here we get the asins that don't have barcodes in the base_format file
        """
        self.missing_asins = self.df[~self.df["ASIN"].isin(self.base_format["ASIN"])].reset_index(drop=True)
        if self.missing_asins.shape[0] == 0:
            self._skip = True
            return
        self.missing_asins = self.missing_asins[["ASIN"]]
        print("###" * 20)
        print("###" * 20)
        for asin in self.missing_asins["ASIN"]:
            print(asin, end=" ")
        print()
        
    def upload_keepa_asins_first(self):
        '''
        Here we upload the name of the keepa file with asins to get the missing barcodes
        '''
        name_file = input("Insert the name of the Keepa file of ASINs: ")
        self.keepa_asin_first = pd.read_excel(f'{name_file}.xlsx')
        
    def process_keepa_asins_first(self):
        '''
        Here we get barcodes that don't exist in base format
        '''
        self.keepa_asin_first = self.keepa_asin_first.rename({'Product Codes: EAN': 'Barcode', 
                                                              'FBA Fees:': "FBA Fee"}, axis=1)
        if self.keepa_asin_first["Barcode"].dtype == "object":
            self.keepa_asin_first['Barcode'] = self.keepa_asin_first['Barcode'].fillna("0").str.split(',').apply(lambda x: [int(ean) for ean in x])
            self.keepa_asin_first = self.keepa_asin_first.explode('Barcode')
            self.keepa_asin_first['Barcode'] = self.keepa_asin_first['Barcode'].astype('Int64')
        self.keepa_asin_first = self.keepa_asin_first[self.keepa_asin_first["Sales Rank: 30 days avg."] < 250_000]
        self.keepa_asin_first = self.keepa_asin_first[self.keepa_asin_first["Sales Rank: Current"] < 250_000]
        self.keepa_asin_first["BB"] = self.keepa_asin_first[["Buy Box: Current", "Buy Box: 30 days avg.", "Buy Box: 90 days avg."]].min(axis=1)
        
    def concat_keepa_barcodes(self):
        '''
        Here we add barcodes and asins from keepa file to baseformat file
        '''
        self.base_format = pd.concat([self.base_format, self.keepa_asin_first[["Barcode", "ASIN", "BB", "FBA Fee"]]], ignore_index=True)
        self.base_format = self.base_format.drop_duplicates(subset=["Barcode", "ASIN"])
   
    def final_formatting(self):
        '''
        Here we make formatting so the file will be ready to anaylize
        '''
        for idx in self.base_format.index:
            sku = self.base_format.loc[idx, "SKU"]
            if sku is np.nan:
                asin = self.base_format.loc[idx, "ASIN"]
                self.base_format.loc[idx, "SKU"] = self.df[self.df["ASIN"] == asin]["Description"].iloc[0]    
        self.base_format = self.base_format.drop(["Description", "prohibited", "Brand", "Sales"], axis=1)
        self.base_format = self.base_format.rename({"SKU": "Description"}, axis=1)
        sales_df = self.df[["ASIN", "Sales"]]
        self.base_format = self.base_format.merge(sales_df, how="left", on="ASIN")
        bb = self.base_format.pop("BB")
        sales = self.base_format.pop("Sales")
        self.base_format.insert(5, "BB", bb)
        self.base_format.insert(7, "Sales", sales)
        self.base_format.insert(10, "Target", None)
        self.base_format.insert(12, "Cost", None)
        
    def name_of_file(self):
        '''
        Here we ask a user what name of the file does he want
        '''
        print("###" * 20)
        print("###" * 20)
        name = input("What name of the file do you want?: ")
        self._name = f'{name} {datetime.now().strftime("%d.%m.%Y")}'
    
    def save_file(self):
        '''
        Here we save the file to excel
        '''
        try:
            self.base_format.to_excel(f'{[self._name, "FINAL"][self._name is None]}.xlsx', index=False)
            return
        except PermissionError:
            print("I can't save the file because it is already opened. Please, close the file!!!")
            name  = input("If you have closed the file, type 'yes': ")
            if name.strip().lower() == "yes":
                self.save_file()
    
    def start(self):
        '''
        Start the whole process
        '''
        self.process_helium()
        self.delete_extra_brands()
        self.merge_baseformat_and_df()
        self.get_missing_asins()
        if self._skip == False:
            self.upload_keepa_asins_first()
            self.process_keepa_asins_first()
            self.concat_keepa_barcodes()
        self.final_formatting()
        self.name_of_file()
        self.save_file()

In [3]:
df = pd.read_csv("Helium_10_Xray_2023-08-31.csv")
base_format = pd.read_excel("Base Format for Profit Check.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'Helium_10_Xray_2023-08-31.csv'